# Tool Retrieval over AI Plugins

This tutorial assumes familiarity with [Natural Language API Toolkits](../../toolkits/examples/openapi_nla.ipynb). NLAToolkits parse whole OpenAPI specs, which can be too large to reasonably fit into an agent's context. This tutorial walks through using a Retriever object to fetch tools.

### First, import dependencies and load the LLM

In [1]:
import re
from typing import Callable, Union, Dict, List, Type

from langchain import OpenAI, LLMChain
from langchain.llms import BaseLLM
from langchain.embeddings.base import Embeddings
from langchain.prompts import StringPromptTemplate
from langchain.requests import Requests
from langchain.agents import AgentExecutor, AgentOutputParser, AgentType, initialize_agent, LLMSingleActionAgent
from langchain.agents.agent_toolkits import NLAToolkit
from langchain.schema import AgentAction, AgentFinish, BaseRetriever
from langchain.tools.plugin import AIPlugin
from langchain.vectorstores.base import VectorStore

In [2]:
# Select the LLM to use. Here, we use text-davinci-003
llm = OpenAI(temperature=0, max_tokens=700) # You can swap between different core LLM's here.

### Next, load the Natural Language API Toolkits

In [3]:
urls = [
    "https://datasette.io/.well-known/ai-plugin.json",
    "https://api.speak.com/.well-known/ai-plugin.json",
    "https://www.wolframalpha.com/.well-known/ai-plugin.json",
    "https://www.zapier.com/.well-known/ai-plugin.json",
    "https://www.klarna.com/.well-known/ai-plugin.json",
    "https://www.joinmilo.com/.well-known/ai-plugin.json",
    "https://slack.com/.well-known/ai-plugin.json",
    "https://schooldigger.com/.well-known/ai-plugin.json",
]

AI_PLUGINS = [AIPlugin.from_url(url) for url in urls]

## Tool Retriever

We will use a vectorstore to create embeddings for each tool description. Then, for an incoming query we can create embeddings for that query and do a similarity search for relevant tools.

In [4]:
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema import Document

In [5]:
embeddings = OpenAIEmbeddings()

In [6]:
# Create the retriever object
from langchain.tools.base import BaseTool

class PluginRouter(BaseTool):
    
    llm: BaseLLM
    retriever: BaseRetriever
    toolkits: Dict[str, NLAToolkit]
    verbose: bool
    name = "AIPluginDelegate"
    
    def _run(self, tool_input: str) -> str:
        """Select the appropriate endpoint and route query there."""
        toolkits = self.retrieve_plugins(tool_input)
        toolkit = toolkits[0]
        agent = initialize_agent(toolkit.get_tools(), llm, verbose=self.verbose)
        return agent.run(tool_input)
    
    async def _arun(self, tool_input: str) -> str:
        raise NotImplementedError("Not implemented.")
        
        
    def retrieve_plugins(self, query: str):
        docs = self.retriever.get_relevant_documents(query)
        return [self.toolkits[d.metadata["plugin_name"]] for d in docs]
    
    @classmethod
    def from_plugins_and_vectorstore_cls(cls, 
                                   llm: BaseLLM,
                                   plugins: List[AIPlugin],
                                   embeddings: Embeddings,
                                   vectorstore_cls: Type[VectorStore],
                                   verbose: bool = True,
                                  ) -> "PluginRouter":
        docs = [
            Document(page_content=plugin.description_for_model, 
                     metadata={"plugin_name": plugin.name_for_model}
                    )
            for plugin in plugins
        ]
        vector_store = vectorstore_cls.from_documents(docs, embeddings)
        toolkits_dict = {plugin.name_for_model: 
                         NLAToolkit.from_llm_and_ai_plugin(llm, plugin) 
                         for plugin in plugins}
        description = f"Useful for completing a single task if it can be handled by an API I am connected to"
        return cls(
            description=description,
            llm=llm,
            retriever=vector_store.as_retriever(),
            toolkits=toolkits_dict,
            verbose=verbose,
        )

In [7]:
router = PluginRouter.from_plugins_and_vectorstore_cls(llm,
                                              AI_PLUGINS,
                                              embeddings,
                                              FAISS,
                                             )

Attempting to load an OpenAPI 3.0.1 spec.  This may result in degraded performance. Convert your OpenAPI spec to 3.1.* spec for better support.
Attempting to load an OpenAPI 3.0.1 spec.  This may result in degraded performance. Convert your OpenAPI spec to 3.1.* spec for better support.
Attempting to load an OpenAPI 3.0.1 spec.  This may result in degraded performance. Convert your OpenAPI spec to 3.1.* spec for better support.
Attempting to load an OpenAPI 3.0.2 spec.  This may result in degraded performance. Convert your OpenAPI spec to 3.1.* spec for better support.
Attempting to load an OpenAPI 3.0.1 spec.  This may result in degraded performance. Convert your OpenAPI spec to 3.1.* spec for better support.
Attempting to load an OpenAPI 3.0.1 spec.  This may result in degraded performance. Convert your OpenAPI spec to 3.1.* spec for better support.
Attempting to load an OpenAPI 3.0.1 spec.  This may result in degraded performance. Convert your OpenAPI spec to 3.1.* spec for better s

In [8]:
router.run("What could I do today with my kiddo?")



> Entering new AgentExecutor chain...
 I should think of something fun and meaningful.
Action: Milo.askMilo
Action Input: What's magic today?
Observation: I'm sorry, I don't understand your query. Please try rephrasing your question or providing more information.
Thought: Maybe I should provide more information.
Action: Milo.askMilo
Action Input: What activities can I do with my child today?
Observation: Today, you can have a fun drawing activity with your child. Give each family member a piece of paper and some drawing supplies. Set a timer for 5 minutes and challenge everyone to draw something related to a chosen theme (e.g., favorite animal, dream vacation). Share and discuss your drawings afterward.
Thought: I now know the final answer.
Final Answer: Today, you can have a fun drawing activity with your child. Give each family member a piece of paper and some drawing supplies. Set a timer for 5 minutes and challenge everyone to draw something related to a chosen theme (e.g., favor

'Today, you can have a fun drawing activity with your child. Give each family member a piece of paper and some drawing supplies. Set a timer for 5 minutes and challenge everyone to draw something related to a chosen theme (e.g., favorite animal, dream vacation). Share and discuss your drawings afterward.'

### Create the Agent

In [9]:
from langchain.agents import load_tools

# Load some other tools
tools = load_tools(["llm-math"], llm=llm)
tools += [router]

In [10]:
agent = initialize_agent(tools, llm, verbose=True)

In [11]:
query = "Help me purchase some Nike shoes. Be sure to price compare."

In [12]:
agent.run(query)



> Entering new AgentExecutor chain...
 I need to find the best deal on Nike shoes.
Action: AIPluginDelegate
Action Input: Search for Nike shoes

> Entering new AgentExecutor chain...
 I should use the Open_AI_Klarna_product_Api.productsUsingGET API to search for Nike shoes
Action: Open_AI_Klarna_product_Api.productsUsingGET
Action Input: Nike shoes
Observation: I found several Nike shoes in the API response. Here is a list of the names and prices: Nike Dunk Low M - Black/White ($82.00), Nike Air Jordan 4 Retro M - Midnight Navy ($284.00), Nike Air Force 1 '07 M - White ($74.00), Nike Dunk Low W - White/Black ($110.00), Nike Air Jordan 1 Retro High M - White/University Blue/Black ($129.00), Nike Air Jordan 11 Retro Cherry - White/Varsity Red/Black ($100.00), Nike Dunk High W - White/Black ($108.00), Nike Air Jordan 1 Retro High OG M - True Blue/Cement Grey/White ($85.00), Nike Air Jordan 5 Retro M - Black/Taxi/Aquatone ($124.00), Nike Court Legacy Lift W ($59.97).
Thought: I now know 

"The best deal on Nike shoes is the Nike Air Force 1 '07 M - White ($74.00)."

## Thank you!